## Adding convolutional layers

In this step we will switch from plain neural networks to convolutional neural networks and see if this makes any difference.

In [65]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

import numpy as np
import math

# fixed random seed to have consistent results
np.random.seed(123)


# dimensions of our images.
img_width, img_height = 300, 300

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 3000
nb_validation_samples = 300
epochs = 5
batch_size = 150

In [66]:
input_shape = (img_height, img_width, 3)

In [67]:
model = Sequential()

# replacing Dense layer with convolution layer
model.add(Conv2D(20, (5, 5), activation = 'relu', input_shape=input_shape)) 
# Think of max pooling as resizing the image to half its dimesion
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(20, (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

In [68]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 296, 296, 20)      1520      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 148, 148, 20)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 144, 144, 20)      10020     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 72, 72, 20)        0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 103680)            0         
_________________________________________________________________
dense_25 (Dense)             (None, 32)                3317792   
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 66        
Total para

**
Notice that number of trainable parameters drastically decreased from 8 millions to 3.4 millions.
**

In [69]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [70]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [71]:
model.fit_generator(
    train_generator,
    steps_per_epoch= math.ceil(nb_train_samples / batch_size),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps= math.ceil(nb_validation_samples/batch_size))

Epoch 1/5
20/20 [==============================] - 52s - loss: 0.8698 - acc: 0.5283 - val_loss: 0.7078 - val_acc: 0.5067

**
Not bad for just two convolution layers, more than 70% now.
Let's do something different in notebook #5 : Tranfer Learning.
**